<a href="https://colab.research.google.com/github/FaizKhalifah/UTS-PPM/blob/main/UTS_NB_dataset4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meng-import dataset

In [ ]:
from google.colab import files
#healthcare-dataset-stroke-data.csv

upload = files.upload()

Saving healthcare-dataset-stroke-data.csv to healthcare-dataset-stroke-data (1).csv


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [ ]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])
data['ever_married']=label_encoder.fit_transform(data['ever_married'])
data['work_type']=label_encoder.fit_transform(data['work_type'])
data['Residence_type']=label_encoder.fit_transform(data['Residence_type'])
data['smoking_status']=label_encoder.fit_transform(data['smoking_status'])

In [ ]:
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,NaN,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.0,2,1


# Membagi dataset menjadi data latih dan data uji

In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

1022
4088


In [ ]:
label_uji = data_uji.pop('stroke')
label_uji

1307    0
734     0
1234    0
4589    0
679     0
       ..
2601    0
2300    0
4944    0
772     0
368     0
Name: stroke, Length: 1022, dtype: int64

In [ ]:
data_uji.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
1307,27572,0,25.0,0,0,0,2,0,92.82,24.1,2
734,3325,1,30.0,0,0,1,3,0,95.01,32.3,3
1234,37053,1,53.0,0,0,1,0,0,78.73,23.3,2
4589,45285,1,37.0,0,0,1,2,1,176.42,39.7,0
679,68131,0,27.0,0,0,0,2,0,149.95,25.9,2


In [ ]:
data_latih.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
2178,16535,0,34.0,0,0,0,2,0,90.15,27.9,1,0
1120,67052,0,36.0,0,0,1,2,1,76.93,21.6,2,0
838,39375,0,40.0,0,0,1,2,0,119.52,34.6,2,0
2242,13380,1,14.0,0,0,0,4,1,111.27,23.2,0,0
2721,23508,0,17.0,0,0,0,1,0,88.57,31.1,2,0


#Menghitung prior

In [ ]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key]=prior[key]/n_data
  return prior

In [ ]:
n_data=len(data_latih)
n_data

4088

In [ ]:
test=Counter(data_latih['stroke'])
test

Counter({0: 3895, 1: 193})

In [ ]:
prior = hitung_prior(data_latih['stroke'])
print(prior)

Counter({0: 0.9527886497064579, 1: 0.04721135029354207})


#Menghitung likelihood

In [ ]:
def hitung_rata2_std_kelas(input_data):
  # Simpan list kolom
  list_columns = input_data.columns[:-1]
  # Simpan nama kolom kelas
  class_column_name = input_data.columns[-1]
  # Simpan list kelas
  list_class = set(input_data[class_column_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
        rata2[(a_class,column)] = input_data.loc[input_data[class_column_name] == a_class][column].mean()
        std[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
  print(rata2)
  print(std)
  return(rata2,std)

In [ ]:
list_columns = data_latih.columns[:-1]
print(list_columns)

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status'],
      dtype='object')


In [ ]:
class_column_name = data_latih.columns[-1]
print(class_column_name)

stroke


In [ ]:
list_class = set(data_latih[class_column_name])
print(list_class)

{0, 1}


In [ ]:
rata2 = {}
std = {}
for column in list_columns:
  for a_class in list_class:
      rata2[(a_class,column)] = data_latih.loc[data_latih[class_column_name] == a_class][column].mean()
      print(rata2)
      std[(a_class,column)] = data_latih.loc[data_latih[class_column_name]==a_class][column].std()
      print(std)
      print("=====")

{(0, 'id'): 36418.05802310655}
{(0, 'id'): 21001.644065054905}
=====
{(0, 'id'): 36418.05802310655, (1, 'id'): 36873.87046632124}
{(0, 'id'): 21001.644065054905, (1, 'id'): 22182.03750149}
=====
{(0, 'id'): 36418.05802310655, (1, 'id'): 36873.87046632124, (0, 'gender'): 0.4102695763799743}
{(0, 'id'): 21001.644065054905, (1, 'id'): 22182.03750149, (0, 'gender'): 0.4924674561985691}
=====
{(0, 'id'): 36418.05802310655, (1, 'id'): 36873.87046632124, (0, 'gender'): 0.4102695763799743, (1, 'gender'): 0.45077720207253885}
{(0, 'id'): 21001.644065054905, (1, 'id'): 22182.03750149, (0, 'gender'): 0.4924674561985691, (1, 'gender'): 0.49886529275033836}
=====
{(0, 'id'): 36418.05802310655, (1, 'id'): 36873.87046632124, (0, 'gender'): 0.4102695763799743, (1, 'gender'): 0.45077720207253885, (0, 'age'): 41.83602567394095}
{(0, 'id'): 21001.644065054905, (1, 'id'): 22182.03750149, (0, 'gender'): 0.4924674561985691, (1, 'gender'): 0.49886529275033836, (0, 'age'): 22.2971346979017}
=====
{(0, 'id'): 

In [ ]:
data_latih.loc[data_latih['stroke']==1]['age']

181    59.0
140    82.0
229    80.0
8      59.0
197    69.0
       ... 
86     61.0
17     75.0
50     76.0
134    68.0
35     82.0
Name: age, Length: 193, dtype: float64

In [ ]:
data_latih.loc[data_latih['stroke']==1]['age'].mean()

68.3539896373057

In [ ]:
data_latih.loc[data_latih['stroke']==1]['age'].std()

12.487710172568018

In [ ]:
import math
def hitung_likelihood_gaussian(data,rata2,std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

#Proses training

In [ ]:
def training_naive_bayes_gaussian(data_latih):
  # simpan daftar kelas
  class_column_name = data_latih.columns[-1]
  # Hitung probility prior
  prior = hitung_prior(data_latih[class_column_name])
  # Hitung mean dan standard deviation
  (rata2,std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

In [ ]:
model = training_naive_bayes_gaussian(data_latih)

{(0, 'id'): 36418.05802310655, (1, 'id'): 36873.87046632124, (0, 'gender'): 0.4102695763799743, (1, 'gender'): 0.45077720207253885, (0, 'age'): 41.83602567394095, (1, 'age'): 68.3539896373057, (0, 'hypertension'): 0.09011553273427471, (1, 'hypertension'): 0.29533678756476683, (0, 'heart_disease'): 0.045699614890885754, (1, 'heart_disease'): 0.18652849740932642, (0, 'ever_married'): 0.6426187419768935, (1, 'ever_married'): 0.8756476683937824, (0, 'work_type'): 2.1817715019255455, (1, 'work_type'): 2.0259067357512954, (0, 'Residence_type'): 0.4980744544287548, (1, 'Residence_type'): 0.5544041450777202, (0, 'avg_glucose_level'): 105.20332220795892, (1, 'avg_glucose_level'): 133.88046632124352, (0, 'bmi'): 28.74730554818158, (1, 'bmi'): 30.48048780487805, (0, 'smoking_status'): 1.3550706033376123, (1, 'smoking_status'): 1.5906735751295338}
{(0, 'id'): 21001.644065054905, (1, 'id'): 22182.03750149, (0, 'gender'): 0.4924674561985691, (1, 'gender'): 0.49886529275033836, (0, 'age'): 22.2971346

#Proses testing

In [ ]:
def testing_naive_bayes_gaussian(model,data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class,1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class]*hitung_likelihood_gaussian(data_uji[column],rata2[(a_class,column)],std[(a_class,column)])
    posterior[a_class] = posterior[a_class] * prior[a_class]
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji

In [ ]:
indeks_uji = 1
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

0
0


In [ ]:
prediksi_total =[]
for indeks in range(data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model,data_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==label_uji))

Total prediksi benar:  884


In [ ]:
print("Tingkat akurasi =",(sum(prediksi_total==label_uji)/data_uji.shape[0])*100)

Tingkat akurasi = 86.49706457925636
